In [32]:
import findspark
findspark.init()

In [33]:
from pyspark import SparkConf, SparkContext

In [36]:
sc.stop()

In [37]:
conf = SparkConf().setMaster("local[4]").setAppName("BroadcastVariablesOps")
sc = SparkContext(conf=conf).getOrCreate()

In [117]:
def read_products():
    products_text_file = open("D:/OneDrive/veribilimi.co/udemy-apache-spark/data/retail_db/products.csv", "r")
    products = products_text_file.readlines()
    product_id_and_name = []
    for line in products:
        product_id = line.split(",")[0]
        product_name = line.split(",")[2]
        product_id_and_name.append((product_id,product_name))
    
    return product_id_and_name
    

In [118]:
products = read_products()

In [120]:
broadcast_product = sc.broadcast(products)

In [122]:
broadcast_product.value

[('productId', 'productName'),
 ('1', 'Quest Q64 10 FT. x 10 FT. Slant Leg Instant U'),
 ('2', "Under Armour Men's Highlight MC Football Clea"),
 ('3', "Under Armour Men's Renegade D Mid Football Cl"),
 ('4', "Under Armour Men's Renegade D Mid Football Cl"),
 ('5', 'Riddell Youth Revolution Speed Custom Footbal'),
 ('6', "Jordan Men's VI Retro TD Football Cleat"),
 ('7', 'Schutt Youth Recruit Hybrid Custom Football H'),
 ('8', "Nike Men's Vapor Carbon Elite TD Football Cle"),
 ('9', 'Nike Adult Vapor Jet 3.0 Receiver Gloves'),
 ('10', "Under Armour Men's Highlight MC Football Clea"),
 ('11', 'Fitness Gear 300 lb Olympic Weight Set'),
 ('12', "Under Armour Men's Highlight MC Alter Ego Fla"),
 ('13', "Under Armour Men's Renegade D Mid Football Cl"),
 ('14', 'Quik Shade Summit SX170 10 FT. x 10 FT. Canop'),
 ('15', "Under Armour Kids' Highlight RM Alter Ego Sup"),
 ('16', 'Riddell Youth 360 Custom Football Helmet'),
 ('17', "Under Armour Men's Highlight MC Football Clea"),
 ('18', "Reebok

# order_item okuma ve rdd oluşturma

In [74]:
order_items_rdd = sc.textFile("D:/OneDrive/veribilimi.co/udemy-apache-spark/data/retail_db/order_items.csv") \
.filter(lambda x: "orderItemName" not in x)

In [75]:
order_items_rdd.take(5)

['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99']

# order_items pair_rdd yapma 

In [76]:
def make_order_items_pair_rdd(line):
    order_item_name = line.split(",")[0]
    order_item_order_id = line.split(",")[1]
    order_item_product_id = int(line.split(",")[2])
    order_item_quantity = line.split(",")[3]
    order_item_sub_total = float(line.split(",")[4])
    order_item_product_price = line.split(",")[5]
    
    return (order_item_product_id, order_item_sub_total)
    

In [77]:
order_item_pair_rdd = order_items_rdd.map(make_order_items_pair_rdd)

In [78]:
order_item_pair_rdd.take(6)

[(957, 299.98),
 (1073, 199.99),
 (502, 250.0),
 (403, 129.99),
 (897, 49.98),
 (365, 299.95)]

# order_items ile broadcast variable olan products birleştirme

In [79]:
order_item_pair_rdd.reduceByKey(lambda x,y: x+y) \
.map(lambda x: (x[1], x[0])) \
.sortByKey(False) \
.map(lambda x: (x[1], x[0])) \
.take(5)

[(1004, 6929653.499999708),
 (365, 4421143.019999639),
 (957, 4118425.419999785),
 (191, 3667633.1999997487),
 (502, 3147800.0)]

In [80]:
sorted_orders = order_item_pair_rdd.reduceByKey(lambda x,y: x+y) \
.map(lambda x: (x[1], x[0])) \
.sortByKey(False) \
.map(lambda x: (x[1], x[0]))

In [ ]:
sorted_orders_with_product_name =

In [125]:
sorted_orders.map(lambda x: (broadcast_product.value[x[0]], x[1])).take(5)

[(('1004', 'Field & Stream Sportsman 16 Gun Fire Safe'), 6929653.499999708),
 (('365', 'Perfect Fitness Perfect Rip Deck'), 4421143.019999639),
 (('957', "Diamondback Women's Serene Classic Comfort Bi"), 4118425.419999785),
 (('191', "Nike Men's Free 5.0+ Running Shoe"), 3667633.1999997487),
 (('502', "Nike Men's Dri-FIT Victory Golf Polo"), 3147800.0)]

In [115]:
from pyspark.context import SparkContext
# sc = SparkContext('local', 'test')
b = sc.broadcast([1, 2, 3, 4, 5])
b.value


[1, 2, 3, 4, 5]

In [116]:

sc.parallelize([0, 0]).flatMap(lambda x: b.value).collect()
b.unpersist()